In [ ]:
## generate glycan profile for a given distribution of enzymes

## TO DO:
## use of function to 
##      1. randomly select the distribution of enzymes in a compartment - DONE
##      2. getting glycan profile for the selected enzyme distribution - DONE
## add extra condition to avoid beyond 2 branches (see if this is required ?)
## elegant way to print the tree                                       - DONE
## also, this generates only 1 product - modify to generate all the products - DONE


# how to generate a graph or tree in python
# https://www.geeksforgeeks.org/generate-graph-using-dictionary-python/

# https://stackoverflow.com/questions/1602934/check-if-a-given-key-already-exists-in-a-dictionary
# https://stackoverflow.com/questions/3199171/append-multiple-values-for-one-key-in-python-dictionary

# find all possible subgraphs of a graph in python
# find all sub-graphs starting from root node in a directed tree/graph in python

In [1]:
## step 1: generate all possible binary combinations of 8 digits (corresponding to 8 enzymes)
#https://stackoverflow.com/questions/14931769/how-to-get-all-combination-of-n-binary-value

import itertools

num_enzymes = 7 #8
# since the first enzyme should be present in the 1st compartment & not in later compartments,
# we can leave this out for now

lst = map(list, itertools.product([0, 1], repeat = num_enzymes))
#lst

In [ ]:
#### mapping file for all enzyme reactions ####

#P/0 -> A/1 (yellow sq) -> B/2 (blue sq)   -> C/3 (red tri)
#  1                    2                3

#p ->   A/1 (yellow sq) -> D/4 (yellow cr) -> B/2 (blue sq)
#                       4                5
                      
#------------------------  D/4 (yellow cr) -> E/5 (brown diamond)
#                                       6
                                     
#------------------------  B/2 (blue sq)   -> D/4 (yellow cr)
#                                       7

#-----  A/1 (yellow sq) -> E/5 (brown rhombus)
#                       8

In [2]:
# create a 2D array with the sequence: enzyme no, 1st substrate, 2nd substrate
map = []
# map.append([1,0,1])   # leave out the first enzyme
map.append([2,1,2])
map.append([4,1,4])
map.append([8,1,5])
map.append([3,2,3])
map.append([7,2,4]) # these 2 enzymes being present in the same compartment will give rise to cyclisation
map.append([5,4,2]) # eliminate correponding combinations from the master_list at the very beginning ?
map.append([6,4,5])

In [3]:
# intialise the sugar as a dictionary
mol = {'1_1':''}  # sugarname_position; the keys are in string format

In [4]:
# foreach item in the above list (of enzyme distribution), if value=1, select the corresponding reaction index from map
# then check if existing glycan list already contains sugars to which the new ones can be appended;
# allow branching where possible : add extra condition to avoid beyond 2 branches

def get_gly(cnt,comp_distr,mol,nodeName):   # cnt,num_enzymes,map
    
    # select enzyme distribution
    #comp_distr = random.choice(lst)
    #comp_distr = [0, 0, 1, 0, 0, 1, 1]   ## comp 1
    #comp_distr = [1, 1, 0, 0, 0, 1, 0]   ## comp 2
    #comp_distr = [1, 1, 0, 1, 1, 0, 0]    ## comp 3
    #print(comp_distr)
    
    # get glycan
    for i in range(num_enzymes): #since, len(comp_distr) = num_enzymes
        if comp_distr[i]==1:
            #print('')    #print(map[i])
            
            # check if the left element of the selected reaction is already present in the glycan chain
            for k in range(1,cnt):  # this indicates values from 1 to cnt-1
                #print(k)   #print(str(map[i][1])+'_'+str(k))
                if (str(map[i][1])+'_'+str(k)) in mol:
                    mol[(str(map[i][1])+'_'+str(k))] = (mol[(str(map[i][1])+'_'+str(k))] + ', '+ str(map[i][2]) + '_' + str(k+1))  
                    #mol[(str(map[i][2])+'_'+str(k+1))] = ''
                    
                    nodeName[(str(map[i][1])+'_'+str(k))] = ''
                    nodeName[(str(map[i][2])+'_'+str(k+1))]= ''
                    
                    if(str(map[i][2])+'_'+str(k+1)) in mol:
                        mol[str(map[i][2])+'_'+str(k+1)] = mol[str(map[i][2])+'_'+str(k+1)]
                    else:
                        mol[(str(map[i][2])+'_'+str(k+1))] = ''
                    cnt = cnt+1
                
    return (mol,cnt,nodeName)


In [ ]:
#######################################  compartment 1  #########################################################

In [5]:
#import random
#comp_distr = random.choice(lst)
comp_distr = [0, 0, 1, 0, 0, 1, 1]
nodeName={}

mol,Cnt,nodeName = get_gly(2,comp_distr,mol,nodeName)     # 2,num_enzymes,map

In [6]:
for key in mol:
    print(key + '->' + mol[key])

5_2->
1_1->, 5_2


In [8]:
################# compartment 2 #####################
#comp_distr = random.choice(lst)
comp_distr = [1, 1, 0, 0, 0, 1, 0] 

mol,Cnt,nodeName = get_gly(Cnt,comp_distr,mol,nodeName)

In [9]:
for key in mol:
    print(key + '->' + mol[key])

4_2->, 2_3
5_2->
2_2->
2_3->
1_1->, 5_2, 2_2, 4_2


In [10]:
################# compartment 3 #####################
#comp_distr = random.choice(lst)
comp_distr = [1, 1, 0, 1, 1, 0, 0]

mol,Cnt,nodeName = get_gly(Cnt,comp_distr,mol,nodeName)

In [11]:
for key in mol:
    print(key + '->' + mol[key])

3_3->
3_4->
2_2->, 3_3, 4_3
2_3->, 3_4, 4_4
5_2->
1_1->, 5_2, 2_2, 4_2, 2_2, 4_2
4_2->, 2_3
4_3->
4_4->


In [ ]:
##############################################################################################
## generate the product profile at the end
# STEPS: 
# clean-up : remove keys with no value assigned
# consider sub-graphs which include only the key 1_1 (root node), but not those with contain only 2_2 or 2_3
#                                    (since these cant start assembly)
# consider all valid combinations


# or, generate the tree using this link : https://www.geeksforgeeks.org/generate-graph-using-dictionary-python/
# so that clean-up is not required

In [12]:
# clean-up : remove keys with no value assigned
clean_mol = dict([(vkey, vdata) for vkey, vdata in mol.iteritems() if(str(vdata).strip()) ])

for key in clean_mol:
    print(key + '->' + clean_mol[key])

4_2->, 2_3
2_2->, 3_3, 4_3
2_3->, 3_4, 4_4
1_1->, 5_2, 2_2, 4_2, 2_2, 4_2


In [13]:
# clean-up : remove ', ' at the beginning by using [2:], split by ', ', & then remove repetitions in the values
for key in clean_mol:
    clean_mol[key] =  ' '.join(list(set(clean_mol[key][2:].split(', ')))) # list(set(mol['1_1'][2:].split(', ')))

for key in clean_mol:
    print(key + '-> ' + clean_mol[key])

4_2-> 2_3
2_2-> 4_3 3_3
2_3-> 3_4 4_4
1_1-> 4_2 5_2 2_2


In [20]:
# count no of nodes in the tree

print(nodeName)
print('')
print('no.of nodes:' + str(len(nodeName)))
#print(nodeName[0])
#print(nodeName[8])

{'3_3': '', '3_4': '', '2_2': '', '2_3': '', '5_2': '', '1_1': '', '4_2': '', '4_3': '', '4_4': ''}

no.of nodes:9


In [ ]:
# print valid sub-graphs starting from root note 1_1

In [15]:
import networkx as nx

In [16]:
# save molecule as graph
g = nx.Graph()

for key in clean_mol:
    # split the nodes attached to each key
    conn_node = clean_mol[key].split(' ')
    
    for j in range(0, len(conn_node)):
        g.add_edge(key, conn_node[j])
        print(str(key)+' '+str(conn_node[j]))

4_2 2_3
2_2 4_3
2_2 3_3
2_3 3_4
2_3 4_4
1_1 4_2
1_1 5_2
1_1 2_2


In [49]:
# https://stackoverflow.com/questions/22180410/how-can-i-extract-all-possible-induced-subgraphs-from-a-given-graph-with-network

import itertools

def get_subgraph(s):
    target = nx.complete_graph(s)
    for sub_nodes in itertools.combinations(g.nodes(),len(target.nodes())):
        subg = g.subgraph(sub_nodes)
        #print(subg.dtype)  #
        if nx.is_connected(subg) and '1_1' in str(subg.edges()):
            print subg.edges()
            #print subg.edges('1_1')
            #(subg.edges()).dtype
            


In [50]:
# print all sub-trees containing '1_1'
for j in range(1, len(nodeName)):  # or, use len(g.nodes)
    get_subgraph(j+1)
    print('')

[('2_2', '1_1')]
[('5_2', '1_1')]
[('1_1', '4_2')]

[('3_3', '2_2'), ('2_2', '1_1')]
[('2_2', '1_1'), ('5_2', '1_1')]
[('2_2', '1_1'), ('1_1', '4_2')]
[('2_2', '4_3'), ('2_2', '1_1')]
[('2_3', '4_2'), ('1_1', '4_2')]
[('5_2', '1_1'), ('1_1', '4_2')]

[('3_3', '2_2'), ('2_2', '1_1'), ('5_2', '1_1')]
[('3_3', '2_2'), ('2_2', '1_1'), ('1_1', '4_2')]
[('3_3', '2_2'), ('2_2', '4_3'), ('2_2', '1_1')]
[('3_4', '2_3'), ('2_3', '4_2'), ('1_1', '4_2')]
[('2_2', '1_1'), ('2_3', '4_2'), ('1_1', '4_2')]
[('2_2', '1_1'), ('5_2', '1_1'), ('1_1', '4_2')]
[('2_2', '4_3'), ('2_2', '1_1'), ('5_2', '1_1')]
[('2_2', '4_3'), ('2_2', '1_1'), ('1_1', '4_2')]
[('2_3', '4_2'), ('5_2', '1_1'), ('1_1', '4_2')]
[('2_3', '4_2'), ('2_3', '4_4'), ('1_1', '4_2')]

[('3_3', '2_2'), ('2_2', '1_1'), ('2_3', '4_2'), ('1_1', '4_2')]
[('3_3', '2_2'), ('2_2', '1_1'), ('5_2', '1_1'), ('1_1', '4_2')]
[('3_3', '2_2'), ('2_2', '4_3'), ('2_2', '1_1'), ('5_2', '1_1')]
[('3_3', '2_2'), ('2_2', '4_3'), ('2_2', '1_1'), ('1_1', '4_2')

In [ ]:
############################################################################################################
####  NOT NEEDED ###########

In [27]:
len(g.nodes)

9

In [15]:
#print("Edges in g: ", g.edges(data=True))


('Edges in g: ', EdgeDataView([('3_3', '2_2', {}), ('3_4', '2_3', {}), ('2_2', '4_3', {}), ('2_2', '1_1', {}), ('2_3', '4_2', {}), ('2_3', '4_4', {}), ('5_2', '1_1', {}), ('1_1', '4_2', {})]))


In [16]:
'''
# https://groups.google.com/forum/#!topic/networkx-discuss/UCmplWkZM3M
def graphtodepth(g,root,depth):
    link_set = set()
    traversed = set()
    rootset = set([root])
    while (depth >= 0) and (rootset):
        newrootset = set()
        for d in rootset:
            traversed.add(d)
            neighbors = g.neighbors(d)
            nextlinks = [(d,n) for n in neighbors]
            newrootset = newrootset.union(set(neighbors)) - traversed
            link_set = link_set.union(nextlinks)
        rootset = newrootset
        depth -= 1
    return nx.Graph(list(link_set)) 


# graphtodepth(g,'1_1',2).edges()
nx.single_source_shortest_path(g,'1_1',cutoff=3)
'''

{'1_1': ['1_1'],
 '2_2': ['1_1', '2_2'],
 '2_3': ['1_1', '4_2', '2_3'],
 '3_3': ['1_1', '2_2', '3_3'],
 '3_4': ['1_1', '4_2', '2_3', '3_4'],
 '4_2': ['1_1', '4_2'],
 '4_3': ['1_1', '2_2', '4_3'],
 '4_4': ['1_1', '4_2', '2_3', '4_4'],
 '5_2': ['1_1', '5_2']}

In [46]:
# https://stackoverflow.com/questions/22180410/how-can-i-extract-all-possible-induced-subgraphs-from-a-given-graph-with-network

import itertools

def get_subgraph(s):
    target = nx.complete_graph(s)
    for sub_nodes in itertools.combinations(g.nodes(),len(target.nodes())):
        subg = g.subgraph(sub_nodes)
        #print(subg.dtype)  #
        if nx.is_connected(subg):  # and '1_1' in str(subg.edges()):
            print subg.edges()
            #print subg.edges('1_1')
            #(subg.edges()).dtype
            


In [18]:
get_subgraph(2)

[('3_3', '2_2')]
[('3_4', '2_3')]
[('2_2', '1_1')]
[('2_2', '4_3')]
[('2_3', '4_2')]
[('2_3', '4_4')]
[('5_2', '1_1')]
[('1_1', '4_2')]


In [19]:
get_subgraph(3)

[('3_3', '2_2'), ('2_2', '1_1')]
[('3_3', '2_2'), ('2_2', '4_3')]
[('3_4', '2_3'), ('2_3', '4_2')]
[('3_4', '2_3'), ('2_3', '4_4')]
[('2_2', '1_1'), ('5_2', '1_1')]
[('2_2', '1_1'), ('1_1', '4_2')]
[('2_2', '4_3'), ('2_2', '1_1')]
[('2_3', '4_2'), ('1_1', '4_2')]
[('2_3', '4_2'), ('2_3', '4_4')]
[('5_2', '1_1'), ('1_1', '4_2')]


In [20]:
get_subgraph(4)

[('3_3', '2_2'), ('2_2', '1_1'), ('5_2', '1_1')]
[('3_3', '2_2'), ('2_2', '1_1'), ('1_1', '4_2')]
[('3_3', '2_2'), ('2_2', '4_3'), ('2_2', '1_1')]
[('3_4', '2_3'), ('2_3', '4_2'), ('1_1', '4_2')]
[('3_4', '2_3'), ('2_3', '4_2'), ('2_3', '4_4')]
[('2_2', '1_1'), ('2_3', '4_2'), ('1_1', '4_2')]
[('2_2', '1_1'), ('5_2', '1_1'), ('1_1', '4_2')]
[('2_2', '4_3'), ('2_2', '1_1'), ('5_2', '1_1')]
[('2_2', '4_3'), ('2_2', '1_1'), ('1_1', '4_2')]
[('2_3', '4_2'), ('5_2', '1_1'), ('1_1', '4_2')]
[('2_3', '4_2'), ('2_3', '4_4'), ('1_1', '4_2')]


In [21]:
get_subgraph(5)

[('3_3', '2_2'), ('2_2', '1_1'), ('2_3', '4_2'), ('1_1', '4_2')]
[('3_3', '2_2'), ('2_2', '1_1'), ('5_2', '1_1'), ('1_1', '4_2')]
[('3_3', '2_2'), ('2_2', '4_3'), ('2_2', '1_1'), ('5_2', '1_1')]
[('3_3', '2_2'), ('2_2', '4_3'), ('2_2', '1_1'), ('1_1', '4_2')]
[('3_4', '2_3'), ('2_2', '1_1'), ('2_3', '4_2'), ('1_1', '4_2')]
[('3_4', '2_3'), ('2_3', '4_2'), ('5_2', '1_1'), ('1_1', '4_2')]
[('3_4', '2_3'), ('2_3', '4_2'), ('2_3', '4_4'), ('1_1', '4_2')]
[('2_2', '1_1'), ('2_3', '4_2'), ('5_2', '1_1'), ('1_1', '4_2')]
[('2_2', '4_3'), ('2_2', '1_1'), ('2_3', '4_2'), ('1_1', '4_2')]
[('2_2', '1_1'), ('2_3', '4_2'), ('2_3', '4_4'), ('1_1', '4_2')]
[('2_2', '4_3'), ('2_2', '1_1'), ('5_2', '1_1'), ('1_1', '4_2')]
[('2_3', '4_2'), ('2_3', '4_4'), ('5_2', '1_1'), ('1_1', '4_2')]


In [22]:
get_subgraph(6)

[('3_3', '2_2'), ('3_4', '2_3'), ('2_2', '1_1'), ('2_3', '4_2'), ('1_1', '4_2')]
[('3_3', '2_2'), ('2_2', '1_1'), ('2_3', '4_2'), ('5_2', '1_1'), ('1_1', '4_2')]
[('3_3', '2_2'), ('2_2', '4_3'), ('2_2', '1_1'), ('2_3', '4_2'), ('1_1', '4_2')]
[('3_3', '2_2'), ('2_2', '1_1'), ('2_3', '4_2'), ('2_3', '4_4'), ('1_1', '4_2')]
[('3_3', '2_2'), ('2_2', '4_3'), ('2_2', '1_1'), ('5_2', '1_1'), ('1_1', '4_2')]
[('3_4', '2_3'), ('2_2', '1_1'), ('2_3', '4_2'), ('5_2', '1_1'), ('1_1', '4_2')]
[('3_4', '2_3'), ('2_2', '4_3'), ('2_2', '1_1'), ('2_3', '4_2'), ('1_1', '4_2')]
[('3_4', '2_3'), ('2_2', '1_1'), ('2_3', '4_2'), ('2_3', '4_4'), ('1_1', '4_2')]
[('3_4', '2_3'), ('2_3', '4_2'), ('2_3', '4_4'), ('5_2', '1_1'), ('1_1', '4_2')]
[('2_2', '4_3'), ('2_2', '1_1'), ('2_3', '4_2'), ('5_2', '1_1'), ('1_1', '4_2')]
[('2_2', '1_1'), ('2_3', '4_2'), ('2_3', '4_4'), ('5_2', '1_1'), ('1_1', '4_2')]
[('2_2', '4_3'), ('2_2', '1_1'), ('2_3', '4_2'), ('2_3', '4_4'), ('1_1', '4_2')]


In [23]:
get_subgraph(7)

[('3_3', '2_2'), ('3_4', '2_3'), ('2_2', '1_1'), ('2_3', '4_2'), ('5_2', '1_1'), ('1_1', '4_2')]
[('3_3', '2_2'), ('3_4', '2_3'), ('2_2', '4_3'), ('2_2', '1_1'), ('2_3', '4_2'), ('1_1', '4_2')]
[('3_3', '2_2'), ('3_4', '2_3'), ('2_2', '1_1'), ('2_3', '4_2'), ('2_3', '4_4'), ('1_1', '4_2')]
[('3_3', '2_2'), ('2_2', '4_3'), ('2_2', '1_1'), ('2_3', '4_2'), ('5_2', '1_1'), ('1_1', '4_2')]
[('3_3', '2_2'), ('2_2', '1_1'), ('2_3', '4_2'), ('2_3', '4_4'), ('5_2', '1_1'), ('1_1', '4_2')]
[('3_3', '2_2'), ('2_2', '4_3'), ('2_2', '1_1'), ('2_3', '4_2'), ('2_3', '4_4'), ('1_1', '4_2')]
[('3_4', '2_3'), ('2_2', '4_3'), ('2_2', '1_1'), ('2_3', '4_2'), ('5_2', '1_1'), ('1_1', '4_2')]
[('3_4', '2_3'), ('2_2', '1_1'), ('2_3', '4_2'), ('2_3', '4_4'), ('5_2', '1_1'), ('1_1', '4_2')]
[('3_4', '2_3'), ('2_2', '4_3'), ('2_2', '1_1'), ('2_3', '4_2'), ('2_3', '4_4'), ('1_1', '4_2')]
[('2_2', '4_3'), ('2_2', '1_1'), ('2_3', '4_2'), ('2_3', '4_4'), ('5_2', '1_1'), ('1_1', '4_2')]


In [24]:
get_subgraph(8)

[('3_3', '2_2'), ('3_4', '2_3'), ('2_2', '4_3'), ('2_2', '1_1'), ('2_3', '4_2'), ('5_2', '1_1'), ('1_1', '4_2')]
[('3_3', '2_2'), ('3_4', '2_3'), ('2_2', '1_1'), ('2_3', '4_2'), ('2_3', '4_4'), ('5_2', '1_1'), ('1_1', '4_2')]
[('3_3', '2_2'), ('3_4', '2_3'), ('2_2', '4_3'), ('2_2', '1_1'), ('2_3', '4_2'), ('2_3', '4_4'), ('1_1', '4_2')]
[('3_3', '2_2'), ('2_2', '4_3'), ('2_2', '1_1'), ('2_3', '4_2'), ('2_3', '4_4'), ('5_2', '1_1'), ('1_1', '4_2')]
[('3_4', '2_3'), ('2_2', '4_3'), ('2_2', '1_1'), ('2_3', '4_2'), ('2_3', '4_4'), ('5_2', '1_1'), ('1_1', '4_2')]


In [25]:
get_subgraph(9)

[('3_3', '2_2'), ('3_4', '2_3'), ('2_2', '4_3'), ('2_2', '1_1'), ('2_3', '4_2'), ('2_3', '4_4'), ('5_2', '1_1'), ('1_1', '4_2')]


In [ ]:
# https://stackoverflow.com/questions/50885343/how-to-extract-all-sub-graphs-containing-a-specified-node-from-a-directed-graph

# https://stackoverflow.com/questions/21739569/finding-separate-graphs-within-a-graph-object-in-networkx
# https://stackoverflow.com/questions/22180410/how-can-i-extract-all-possible-induced-subgraphs-from-a-given-graph-with-network
# https://stackoverflow.com/questions/13914920/networkx-extract-the-connected-component-containing-a-given-node-directed-grap
# https://stackoverflow.com/questions/18643789/how-to-find-subgraphs-in-a-directed-graph-without-converting-to-undirected-graph
# https://stackoverflow.com/questions/40284774/efficient-way-for-finding-all-the-complete-subgraphs-of-a-given-graph-python


# https://networkx.github.io/documentation/stable/auto_examples/subclass/plot_printgraph.html


# https://stackoverflow.com/questions/16150557/networkxcreating-a-subgraph-induced-from-edges
# https://stackoverflow.com/questions/47892944/python-networkx-find-a-subgraph-in-a-directed-graph-from-a-node-as-root?rq=1
# https://stackoverflow.com/questions/36488758/networkx-find-root-node-for-a-particular-node-in-a-directed-graph
# https://stackoverflow.com/questions/18843247/find-the-root-in-a-given-graph

# google:
# depth first search in networkx
# find all subgraphs containing a specified node in a directed graph in python
# networkX root node to leaf node
# how to extract all subgraphs containing a specified node from a given directed graph
# stackoverflow find sub graph with root node from tree

In [ ]:
'''
# https://stackoverflow.com/questions/7134742/python-yield-all-paths-from-leaves-to-root-in-a-tree
# https://anandology.com/python-practice-book/iterators.html
# https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/

def paths(self, acc=[]):
    if self.is_leaf():
        yield [self.node]+acc

    for child in self.children:
        for leaf_path in child.paths([self.node]+acc): # these two
            yield leaf_path                            # lines do that  
            
'''